### Data Set Overview

+ The dataset is taken from **kaggle** and contains details of the **used cars in germany** which are on sale on **ebay**.
+ The dataset is not clean and hence a lot of data cleaning is carried out. For e.g. prices where too high which are replaced by the median and outliers are removed accordingly. 
 + Also vehicles whose registration year was **_greater than 2016_ and _less than 1890_** were removed from the dataset as this data is inconsistense and would yield incorrect results.
 + As per project instruction, the initial data analysis and cleansing part has been kept minimal and EDA part has been more elaborative.

### Sample Data

dateCrawled | name | seller | offerType | price | abtest | vehicleType | yearOfRegistration | gearbox | powerPS | model | kilometer | monthOfRegistration | fuelType | brand | notRepairedDamage | dateCreated | nrOfPictures | postalCode | lastSeen
--- | --- | --- | --- | --- | --- | --- | --- | --- |--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | ---
2016-03-24 11:52:17 | Golf_3_1.6 | privat | Angebot | 480 | test | nan | 1993 | manuell | 0 | golf | 150000 | 0 | benzin | volkswagen | nan | 2016-03-24 00:00:00 | 0 | 70435 | 2016-04-07 03:16:57
2016-03-24 10:58:45 | A5_Sportback_2.7_Tdi | privat | Angebot | 18300 | test | coupe | 2011 | manuell | 190 | nan | 125000 | 5 | diesel | audi | ja | 2016-03-24 00:00:00 | 0 | 66954 | 2016-04-07 01:46:50
2016-03-14 12:52:21 | Jeep_Grand_Cherokee_"Overland" | privat | Angebot | 9800 | test | suv | 2004 | automatik | 163 | grand | 125000 | 8 | diesel | jeep | nan | 2016-03-14 00:00:00 | 0 | 90480 | 2016-04-05 12:47:46

### 1. Initial setup

In [32]:
# importing required libraries
import os
import subprocess
import stat
import pandas_profiling
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import thinkstats2
import thinkplot
from datetime import datetime
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
sns.set(style="white")

# ensuring graphs are displayed within same notebook, rather than new window
%matplotlib inline

### 2. Data Ingestion

In [33]:
# get absolute path till the raw data file
abs_path = os.getcwd()
data_path = abs_path + "/RawData/autos.csv"

# reading csv into raw dataframe
rawdf = pd.read_csv(data_path,encoding="latin-1")

C:\Users\Jgars\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 3. Data Profiling

In [34]:
# Create a data profile to plan data cleansing
pandas_profiling.ProfileReport(rawdf)

C:\Users\Jgars\AppData\Roaming\Python\Python37\site-packages\pandas_profiling\model\correlations.py:124: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'The internally computed table of expected frequencies has a zero element at (0, 0).')
  correlation_name=correlation_name, error=error


The package in the previous step creates data profile and handles a lot of descriptive statistics like `mean`, `median`, `mode`, `quantile` and even correlation and distribuion. This takes a lot of individual steps away from the process.

The following _Data Cleansing_ section is largely driven by the data profile created already.

### 4. Data Cleansing

Remove irrelevant or duplicate data

In [35]:
# Since there are 4 duplicate rows in the dataset, getting rid of the duplicates
df = rawdf.drop_duplicates().copy()

# dropping rows that are inconsistent with the dataset based on year of registration
df = df[df["yearOfRegistration"].between(1890, 2016, inclusive=True)]

# deleting the column noOfPictures since all of them are Zero
del df["nrOfPictures"]

Handle `null` or missing values

In [36]:
# setting the NaN fuelType types to other
df["fuelType"].fillna("other",inplace=True)

# setting the NaN gearbox types to Unspecified
df["gearbox"].fillna("Unspecified", inplace=True)

# setting nan in model column to Other
df["model"].fillna("Other",inplace=True)

# setting nan in notRepairedDamage to other
df["notRepairedDamage"].fillna("other",inplace=True)

# changing the vehicleType from NaN to Others
df["vehicleType"].fillna("Other", inplace=True)

Handle language locale

In [37]:
# Since the dataset is German, converting some of the German words in English

# changing values of offerType Gesuch to Request and Angebot to Offer
df["offerType"] = df["offerType"].map({'Gesuch':"Request",'Angebot':'Offer'})

# Changing german names to english readable format
df["fuelType"] = df["fuelType"].map({'benzin':'Gasoline','diesel':'Diesel','other':'Other','lpg':'Lpg','hybrid':'Hybrid','cng':'Cng','elektro':'Electric'})

# Changing german names to english readable format
df["notRepairedDamage"] = df["notRepairedDamage"].map({'other':'Other','ja':'Yes','nein':'No'})

Find the interquartile range for `vehicleType` and remove outliers.

In [38]:
# Mean of the prices of all the vehicle types
_median = df.groupby("vehicleType")["price"].median()

# 75th percentile of the prices of all the vehicles types
_quantile75 = df.groupby("vehicleType")["price"].quantile(0.75)

# 25th percentile of the prices of all the vehicles types
_quantile25 = df.groupby("vehicleType")["price"].quantile(0.25)

# Calculating the value of the prices of each vehicle type above which all the values are outliers
iqr = (_quantile75 - _quantile25)*1.5 + _median
iqr

vehicleType
Other             3708.0
andere            8434.0
bus              13948.5
cabrio           21400.0
coupe            24300.0
limousine        12801.5
small_car         5775.0
station_wagon    13076.5
suv              28800.0
Name: price, dtype: float64

In [39]:
# Removing the outliers as per the logic above
df = df[((df["vehicleType"] == "andere") & (df["price"] <= 8429)) |
        ((df["vehicleType"] == "Other") & (df["price"] <= 3708)) |
        ((df["vehicleType"] == "suv") & (df["price"] <= 28800)) |
        ((df["vehicleType"] == "kombi") & (df["price"] <= 13076)) |
        ((df["vehicleType"] == "bus") & (df["price"] <= 13948)) |
        ((df["vehicleType"] == "cabrio") & (df["price"] <= 21400)) |
        ((df["vehicleType"] == "limousine") & (df["price"] <= 12801)) |
        ((df["vehicleType"] == "coupe") & (df["price"] <= 24300)) |
        ((df["vehicleType"] == "kleinwagen") & (df["price"] <= 5775))]

Handling dates

In [40]:
# Create column for year of creation
df["yearOfCreation"] = df['dateCreated'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S').date().strftime('%Y'))

# Create column for year of crawling
df["yearCrawled"] = df['dateCrawled'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S').date().strftime('%Y'))

# Create column for month of creation
df["monthOfCreation"] = df['dateCreated'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S').date().strftime('%B'))

# Create column for month of crawling
df["monthCrawled"] = df['dateCrawled'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S').date().strftime('%B'))

# Create column for number of days seen online
from_date = df['dateCreated'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S').date())
to_date = df['lastSeen'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S').date())
no_days = abs((to_date-from_date))
no_days = (no_days / np.timedelta64(1, 'D')).astype(int)
df["NoOfDaysOnline"] = no_days

# Create column for number of hrs seen online
hrs_from = df['dateCreated'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S').time().strftime('%H'))
hrs_to = df['lastSeen'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S').time().strftime('%H'))
total_hrs = (hrs_to.astype(int) - hrs_from.astype(int))
df["NoOfHrsOnline"] = total_hrs

# Map the values on monthOfRegistration column from 1-12 
df["monthOfRegistration"].replace([0,12],[1,11],inplace=True)

# Calculate age of vehicle in years
df["yearsOld"] = 2016 - df["yearOfRegistration"]

# Calculate age of vehicle in months
df["monthsOld"] = 12 - df["monthOfRegistration"]

# Map the values on monthOfRegistration column from 1-12 to Jan to Dec
months = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov"]
df["monthOfRegistration"].replace([1,2,3,4,5,6,7,8,9,10,11],months,inplace=True)

Find correlation with price.

In [0]:
corr = df.corr()

sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

## Exploratory Data Analysis

### 1. Histograms

I chose the car price and 4 other features that have the strongest correlation in the above plot.

#### Price

In [0]:
plt.hist(df.price, bins=40)
plt.title("Price")

A lot of records has price as zero and very few has price greater than 20,000, with 28,800 being the most

#### Year of Registration

In [0]:
plt.hist(df.yearOfRegistration, bins=40)
plt.title("Year Of Registration")

We can see that even after filtering the dataset between 1890 and 2016, we still have an outlier at 1910. Most of the listed cars are registered between 1990 and 2015.

#### Years Old

In [0]:
plt.hist(df.yearsOld, bins=40)
plt.title("Years Old")

Most of the cars are between 10 to 20 years old, with an outlier of 106 years old car. 

#### Kilometer

In [0]:
plt.hist(df.kilometer, bins=10)
plt.title("Kilometer")

Since the data is of used cars, most of the cars have milleage acquired, but most of the cars have between 12,000 to 15,000 kilometer under the belt.

In [0]:
plt.hist(df.NoOfDaysOnline, bins=60)
plt.title("No Of Days Online")

Majority of the cars came online very recently, under 10 days. But there is an outlier that has been online for 384 days.

### 2. Probability Mass Function (PMF)

I am going to divide the data set into two part, based on price. One part with price higher than the median value and another with price lower than the median value.

In [0]:
medianprice = df.price.median()

df_highprice = df.price[df['price'] >= medianprice]
df_lowprice = df.price[df['price'] < medianprice]

df_lowprice = df_lowprice.astype(float)

In [0]:
bins = 100

plt.hist(df_highprice, bins, alpha=0.9, label='>= median price', color='skyblue')
plt.hist(df_lowprice, bins, alpha=0.5, label='lower than median price', color='red')
plt.legend(loc='upper right')
plt.show()

In [0]:
# Another way
pmf = df_highprice.value_counts().sort_index() / len(df_highprice)
pmf.head()

Since _pmf_ is normalized, total probability would be __1__.

In [0]:
pmf.sum()

In [0]:
# Using the thinkstats2 module
thinkstats2.Pmf(df_highprice)

### 3. Cumulative Distribution Function (CDF)

I am using the _kilometer_ variable from the cars data to show how the kilometer reading of the cars is distributed across the data.

In [0]:
cdf = thinkstats2.Cdf(df.kilometer)
thinkplot.Cdf(cdf)
thinkplot.Show(xlabel='Kilometer Reading', ylabel='CDF')

It looks like about 20% cars has run under 100000 kilometer, and about 30% cars has run under 120000 kilometer. This could be validated from the histogram of kilometer plotted earlier where we can see that the majority of cars fall under the ">120000" category. 
Common values appear as steep or vertical sections of the CDF; there are fewer values below 100000 kilometer, so the CDF in this range is flatter.

### 4. Analytical Distribution

For this, I am generating the normal probability plot for car price. It plots a gray line that represents the model and a blue line that represents the data.

In [0]:
def MakeNormalPlot(weights):
    mean = weights.mean()
    std = weights.std()
    xs = -2, 2
    fxs, fys = thinkstats2.FitLine(xs, inter=mean, slope=std)
    thinkplot.Plot(fxs, fys, color='gray', label='model')
    xs,ys = thinkstats2.NormalProbability(weights)
    thinkplot.Plot(xs,ys, label='Model')

In [0]:
MakeNormalPlot(df.price)

The data is closer to the model near the mean but deviates at the ends, meaning the expensive cars are more expensive than predicted. The data does not have any negative values as predicted by the model.

### 5. Scatter Plots

#### 1. Examining relationship between car price and the power of the car in PS. 

Excluding low powerPS values

In [0]:
thinkplot.Scatter(df.powerPS[df['powerPS'] >= 800], df.price[df['powerPS'] >= 800])
thinkplot.Show(xlabel='powerPS', ylabel='Car Price')

In [0]:
# Pearson coeficient
pearsonr(df.powerPS[df['powerPS'] >= 800], df.price[df['powerPS'] >= 800])

From the plot and the coefficient, looks like powerPS and Car price has a weak positive correlation.

#### 2. Examining relationship between car's age and price

Excluding newer cars since their price is expected to be higher.

In [0]:
thinkplot.Scatter(df.yearsOld[df['yearsOld'] >= 6], df.price[df['yearsOld'] >= 6])
thinkplot.Show(xlabel='Years Old', ylabel='Car Price')

In [0]:
# Pearson coeficient
pearsonr(df.yearsOld[df['yearsOld'] >= 6], df.price[df['yearsOld'] >= 6])

Car price has a declining relationship with the age of the car. From the data, we can say that the age is one of the factors that causes the price to change. We can see some outliers for cars older than 35 years who has high price.

### 6. Hypothesis Testing

Test correlation.

In [0]:
class CorrelationPermute(thinkstats2.HypothesisTest):

    def TestStatistic(self, data):
        xs, ys = data
        test_stat = abs(thinkstats2.Corr(xs, ys))
        return test_stat

    def RunModel(self):
        xs, ys = self.data
        xs = np.random.permutation(xs)
        return xs, ys

In [0]:
# Section data off into only the two columns that we want to test
data = df.price.values, df.kilometer.values

ht = CorrelationPermute(data)
ht.PValue()

_p-value_ of 0 indicates a failure to reject the null hypothesis at the 5% significance level.

### 7. Regression Analysis

Regression Analysis of one dependent and multiple explanatory variables.

In [0]:
y = df['price'] #value we are predicting
x = df[['kilometer', 'powerPS', 'yearsOld', 'NoOfDaysOnline']] #explanatory variables

X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.20, random_state=0) #split the data 80/20

model = LinearRegression()
model.fit(X_train,y_train)

In [0]:
coeff_df = pd.DataFrame(model.coef_, x.columns, columns=['Coefficient'])  
coeff_df 

The _regression coefficient_ is the constant that represents the rate of change of one variable (price) as a function of changes in the others (kilometer, powerPS, yearsOld, NoOfDaysOnline); it is the slope of the regression line. In this case, Each coefficient estimates the change in the mean response per unit increase in price when all other predictors are held constant.

__** End of Code **__